### Imports etc

In [1]:
import sys
from pathlib import Path
import pandas as pd
import seaborn as sns

REPO_PATH = Path.home() / "workspace/jetpointnet"
SCRIPT_PATH = REPO_PATH / "python_scripts"
sys.path.append(str(SCRIPT_PATH))

import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# from mpl_toolkits.axes_grid1 import make_axes_locatable
from jets_training.models.JetPointNet import PointNetSegmentation
from jets_training.jets_visualize import generate_images_and_metadata
from jets_training.jets_train import (
    TRAIN_INPUTS,
    MAX_SAMPLE_LENGTH,
    baseline_configuration,
    EXPERIMENT_NAME,
)
OUTPUT_ACTIVATION_FUNCTION = baseline_configuration['OUTPUT_ACTIVATION_FUNCTION']




2024-07-18 20:37:56.322818: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 20:37:57.645216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 20:37:57.647804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 20:37:57.932735: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 20:37:58.472771: I tensorflow/core/platform/cpu_feature_guar

Logged in as jhimmens


# Generate Event Images

In [2]:
model_path = "/home/jhimmens/workspace/jetpointnet/models/rev_4/collected_data/PointNet_best_name=valiant-plasma-941.keras"
sets_to_visualize = ["JZ1", 'JZ2', "JZ3", 'JZ4', "rho", 'delta']
max_events_per_set = 10_000
max_images_per_set = 10

MODEL_NAME = model_path.split('/')[-1]
VISUAL_PATH = REPO_PATH / "visualizations" / EXPERIMENT_NAME / MODEL_NAME

model = PointNetSegmentation(MAX_SAMPLE_LENGTH, 
                             num_features=len(TRAIN_INPUTS), 
                             num_classes=1, 
                             output_activation_function=OUTPUT_ACTIVATION_FUNCTION,
                             model_version=1)

model.load_weights(model_path)

generate_images_and_metadata(sets_to_visualize, model, max_events_per_set, max_images_per_set, VISUAL_PATH)

2024-07-18 20:38:10.049496: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-18 20:38:10.049643: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: triumf-ml2
2024-07-18 20:38:10.049656: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: triumf-ml2
2024-07-18 20:38:10.049922: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: INVALID_ARGUMENT: expected %d.%d, %d.%d.%d, or %d.%d.%d.%d form for driver version; got "1"
2024-07-18 20:38:10.050015: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.67.0


  0%|          | 0/6 [00:00<?, ?it/s]

JZ1
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val


JZ1: 0it [00:00, ?it/s]

JZ2
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val


...

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/JZ4/user.mswiatlo.39955735._000005.mltree.root_chunk_0_val.parquet.npz
224/224 [==============================] - 46s 205ms/step
saving: event=000004111_track=000
saving: event=000004111_track=001
saving: event=000004111_track=002
saving: event=000004111_track=003
saving: event=000004111_track=004
saving: event=000004111_track=005
saving: event=000004111_track=006
saving: event=000004111_track=007
saving: event=000004111_track=008
saving: event=000004111_track=009
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/JZ4/user.mswiatlo.39955735._000543.mltree.root_chunk_0_val.parquet.npz
232/232 [==============================] - 51s 221ms/step
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_

rho: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/rho/user.mswiatlo.40097513._000008.mltree.root_chunk_26_val.parquet.npz
7/7 [==============================] - 1s 172ms/step
saving: event=000927048_track=000
saving: event=000927032_track=000
saving: event=000927078_track=000
saving: event=000927076_track=000
saving: event=000927083_track=000
saving: event=000927095_track=000
saving: event=000927082_track=000
saving: event=000927099_track=000
saving: event=000927099_track=001
saving: event=000927099_track=002
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/rho/user.mswiatlo.40097513._000014.mltree.root_chunk_1_val.parquet.npz
7/7 [==============================] - 2s 196ms/step
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=80

delta: 0it [00:00, ?it/s]

/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/delta/user.mswiatlo.40097496._000002.mltree.root_chunk_27_val.parquet.npz
8/8 [==============================] - 1s 146ms/step
saving: event=000368969_track=000
saving: event=000368959_track=000
saving: event=000368975_track=000
saving: event=000368931_track=000
saving: event=000368997_track=000
saving: event=000368973_track=000
saving: event=000368893_track=000
saving: event=000368790_track=000
saving: event=000368790_track=001
saving: event=000368771_track=000
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLen=800_EScale=1/val/delta/user.mswiatlo.40097496._000008.mltree.root_chunk_7_val.parquet.npz
8/8 [==============================] - 1s 118ms/step
/fast_scratch_1/atlas/pflow/jhimmens_working_files/pnet_data/processed_files/collected_data/rev_4/SavedNpz/deltaR=0.2_maxLe